What this code did :

*   Downloads Oct, Nov, Dec 2023 NYC Yellow Taxi data.
*   Cleans the dataset (removes wrong trips, fills NaNs, fixes types).
*   Creates a new clean DataFrame called df_clean.
*   Checks for any missing values column by column.

In [4]:
import pandas as pd

# List of months to combine
months = ["2023-10", "2023-11", "2023-12"]

dfs = []
for m in months:
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{m}.parquet"
    print("📥 Downloading:", url)
    df_tmp = pd.read_parquet(url)
    print(f"   Rows in {m}: {len(df_tmp)}")
    dfs.append(df_tmp)

# Combine datasets
df_big = pd.concat(dfs, ignore_index=True)

print("\n✅ Combined dataset")
print("Total rows:", len(df_big))
print("Total columns:", df_big.shape[1])

# Preview first few rows
df_big.head()



📥 Downloading: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-10.parquet
   Rows in 2023-10: 3522285
📥 Downloading: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-11.parquet
   Rows in 2023-11: 3339715
📥 Downloading: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-12.parquet
   Rows in 2023-12: 3376567

✅ Combined dataset
Total rows: 10238567
Total columns: 19


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-10-01 00:16:44,2023-10-01 00:16:49,1.0,0.00,1.0,N,168,168,2,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0
1,1,2023-10-01 00:23:24,2023-10-01 00:23:47,1.0,0.00,1.0,N,168,168,2,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0
2,1,2023-10-01 00:21:18,2023-10-01 00:27:31,1.0,0.90,1.0,N,161,186,1,6.5,3.5,0.5,2.9,0.0,1.0,14.4,2.5,0.0
3,1,2023-10-01 00:17:39,2023-10-01 00:17:47,0.0,0.00,1.0,N,255,255,3,3.0,1.0,0.5,0.0,0.0,1.0,5.5,0.0,0.0
4,2,2023-10-01 00:16:15,2023-10-01 00:22:57,2.0,1.41,1.0,N,151,239,1,10.0,1.0,0.5,3.0,0.0,1.0,18.0,2.5,0.0


In [5]:
# Check if there are any NaN values in the dataset
df_big.isnull().sum()


,0
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,467607
trip_distance,0
RatecodeID,467607
store_and_fwd_flag,467607
PULocationID,0
DOLocationID,0
payment_type,0


In [6]:
# Work on a copy
df = df_big.copy()

In [7]:
print("Initial shape:", df.shape)

Initial shape: (10238567, 19)


In [9]:
# Count of missing values per column
nan_counts = df_big.isnull().sum()

# Percentage of missing values per column
nan_percent = (nan_counts / len(df_big)) * 100

# Show only columns that have missing values
nan_summary = nan_percent[nan_percent > 0].sort_values(ascending=False)

print("Percentage of missing values per column:")
print(nan_summary)

Percentage of missing values per column:
passenger_count         4.567114
RatecodeID              4.567114
store_and_fwd_flag      4.567114
congestion_surcharge    4.567114
Airport_fee             4.567114
dtype: float64


In [10]:
# Average percentage of missing values across all columns
overall_missing_percent = nan_summary.mean()
print("\n Average percentage of missing data (only in columns with NaNs):", round(overall_missing_percent, 2), "%")


 Average percentage of missing data (only in columns with NaNs): 4.57 %


In [11]:
# Parse datetimes (robust) if present
for col in ["tpep_pickup_datetime", "tpep_dropoff_datetime"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

In [12]:
print("Step 2 Datetime columns converted")
print(df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]].head(3))

Step 2 Datetime columns converted
  tpep_pickup_datetime tpep_dropoff_datetime
0  2023-10-01 00:16:44   2023-10-01 00:16:49
1  2023-10-01 00:23:24   2023-10-01 00:23:47
2  2023-10-01 00:21:18   2023-10-01 00:27:31


In [14]:
# Create trip duration (minutes)
if {"tpep_pickup_datetime","tpep_dropoff_datetime"}.issubset(df.columns):
    df["duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
else:
    # If those columns aren't present, make a placeholder so filters won't crash
    df["duration_min"] = np.nan

In [16]:
print("Created duration_min column")
print(df["duration_min"].describe())

Created duration_min column
count    1.023857e+07
mean     1.543930e+01
std      8.828303e+03
min     -2.824830e+07
25%      7.983333e+00
50%      1.338333e+01
75%      2.200000e+01
max      7.188317e+03
Name: duration_min, dtype: float64


In [18]:
# Basic sanity filters (tune thresholds as needed)
#    - duration between 1 and 180 minutes
#    - nonnegative & non-extreme distance
#    - positive & reasonable fare/total amounts
mask = pd.Series(True, index=df.index)

if "duration_min" in df.columns:
    mask &= df["duration_min"].between(1, 180, inclusive="both")

if "trip_distance" in df.columns:
    mask &= (df["trip_distance"] >= 0) & (df["trip_distance"] < 100)

for amt_col, upper in [("fare_amount", 1000), ("total_amount", 1500)]:
    if amt_col in df.columns:
        mask &= df[amt_col].notna() & (df[amt_col] > 0) & (df[amt_col] < upper)

if "passenger_count" in df.columns:
    df["passenger_count"] = df["passenger_count"].fillna(1)
    mask &= df["passenger_count"].between(1, 6, inclusive="both")

before_rows = len(df)
df = df[mask].reset_index(drop=True)
after_rows = len(df)
print("Applied sanity filters")
print(f"Rows before: {before_rows:,} | after: {after_rows:,} | removed: {before_rows - after_rows:,}")

Applied sanity filters
Rows before: 9,880,959 | after: 9,880,959 | removed: 0


In [20]:
# Handle NaNs (numeric → median, categorical → mode)
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
cat_cols = [c for c in df.columns if c not in num_cols]

# Fill numeric NaNs
for c in num_cols:
    if df[c].isna().any():
        df.loc[df[c].isna(), c] = df[c].median()

# Fill categorical NaNs
for c in cat_cols:
    if df[c].isna().any():
        try:
            df.loc[df[c].isna(), c] = df[c].mode().iloc[0]
        except Exception:
            df.loc[df[c].isna(), c] = "Unknown"

print("Handled NaN values")
print("Columns still with NaN:", df.columns[df.isna().any()].tolist())

Handled NaN values
Columns still with NaN: []


In [21]:
df_clean = df.copy()
print("Data cleaned successfully")
print("Shape after cleaning:", df_clean.shape)

Data cleaned successfully
Shape after cleaning: (9880959, 20)


In [22]:
# Final clean frame
df_clean = df.copy()
print("Step 7 Final clean data ready ")
print("Final shape:", df_clean.shape)
print("Any NaNs left?", df_clean.isnull().values.any())

Step 7 Final clean data ready 
Final shape: (9880959, 20)
Any NaNs left? False


In [23]:
print("Total rows:", len(df_clean))
print("Total columns:", df_clean.shape[1])

Total rows: 9880959
Total columns: 20


In [24]:
df_clean.isnull().sum()

,0
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
RatecodeID,0
store_and_fwd_flag,0
PULocationID,0
DOLocationID,0
payment_type,0


In [25]:
print("Before cleaning (df_big):", len(df_big))
print("After cleaning (df_clean):", len(df_clean))
print("Rows removed:", len(df_big) - len(df_clean))

Before cleaning (df_big): 10238567
After cleaning (df_clean): 9880959
Rows removed: 357608


In [26]:
# Save the cleaned dataset to a CSV file
file_path = "/content/cleaned_taxi_data.csv"
df_clean.to_csv(file_path, index=False)

# Let the user download it
from google.colab import files
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
# Save both datasets directly to Google Drive

# 1️Set the paths in your Google Drive
cleaned_path = "/content/drive/MyDrive/cleaned_taxi_data.csv"
raw_path = "/content/drive/MyDrive/NYC_Taxi_Before_Cleaning.csv"

# Save the cleaned dataset
df_clean.to_csv(cleaned_path, index=False)
print("Cleaned dataset saved to Google Drive as 'cleaned_taxi_data.csv'")

# Save the combined (raw) dataset before cleaning
df_big.to_csv(raw_path, index=False)
print("Raw (before cleaning) dataset saved to Google Drive as 'NYC_Taxi_Before_Cleaning.csv'")


Cleaned dataset saved to Google Drive as 'cleaned_taxi_data.csv'
Raw (before cleaning) dataset saved to Google Drive as 'NYC_Taxi_Before_Cleaning.csv'
